In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

# Load and clean dataset
books = pd.read_csv("books.csv")
ratings = pd.read_csv("ratings.csv")

In [ ]:
# Remove users and books below the threshold
user_counts = ratings["user_id"].value_counts()
book_counts = ratings["isbn"].value_counts()

filtered_users = user_counts[user_counts >= 200].index
filtered_books = book_counts[book_counts >= 100].index

filtered_ratings = ratings[ratings["user_id"].isin(filtered_users) & ratings["isbn"].isin(filtered_books)]

In [ ]:
# Merge with books dataset
combined = filtered_ratings.merge(books, on="isbn").drop_duplicates()

In [ ]:
# Build the Nearest Neighbors model
model = NearestNeighbors(metric="cosine", algorithm="brute")
book_features = combined.pivot(index="isbn", columns="user_id", values="rating").fillna(0)
model.fit(book_features)

In [ ]:
# Define the recommendation function
def get_recommends(book_title):
    book_index = combined[combined["title"] == book_title].index[0]
    distances, indices = model.kneighbors(book_features.iloc[book_index].values.reshape(1, -1), n_neighbors=6)

    recommendations = [(combined.iloc[idx]["title"], dist) for idx, dist in zip(indices[0][1:], distances[0][1:])]
    return [book_title, recommendations]

In [ ]:
# Example usage:
print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()